# 6. Exact Inference Using Graphical Models

## 1) Complexity of inference

- MAP queries
Maxinum a posteriori

### Real-world issues


## 2) Using the variable Elimination algorithm

brute force

Variable Elimination

In [1]:
from libpgm.graphskeleton import GraphSkeleton
from libpgm.nodedata import NodeData
from libpgm.discretebayesiannetwork import DiscreteBayesianNetwork
from libpgm.tablecpdfactor import TableCPDFactor
import itertools
import pandas as pd
from libpgm.tablecpdfactorization import TableCPDFactorization
def loadbn(jsonpath):
   nd = NodeData()
   skel = GraphSkeleton()
   nd.load(jsonpath)
   skel.load(jsonpath)
   skel.toporder()
   bn = DiscreteBayesianNetwork(skel, nd)
   return bn
bn=loadbn("asia1.txt")


In [2]:
#a method that prints the distribution as a table.
def printdist(jd,bn,normalize=False):
    x=[bn.Vdata[i]["vals"] for i in jd.scope]
    zipover=[i/sum(jd.vals) for i in jd.vals] if normalize else jd.vals
    #creates the cartesian product
    k=[a + [b] for a,b in zip([list(i) for i in itertools.product(*x[::-1])],zipover)]
    df=pd.DataFrame.from_records(k,columns=[i for i in reversed(jd.scope)]+['probability'])
    return df
#instantiate TableCPDs for all the nodes and multiply them. The result is in the factor that calls the methods.
jc=TableCPDFactor("asia",bn)
[jc.multiplyfactor(TableCPDFactor(i,bn)) for i in bn.V if i != "asia"]
df=printdist(jc,bn,normalize=True)
print ("values in joint distribution ",len(jc.vals))
#print the first few values in the table
df.head()

values in joint distribution  256


,dysp,xray,either,tub,lung,bronc,smoke,asia,probability
0,yes,yes,yes,yes,yes,yes,yes,yes,1.323000e-05
1,yes,yes,yes,yes,yes,yes,yes,no,2.619540e-04
2,yes,yes,yes,yes,yes,yes,no,yes,6.615000e-07
3,yes,yes,yes,yes,yes,yes,no,no,1.309770e-05
4,yes,yes,yes,yes,yes,no,yes,yes,6.860000e-06


### Marginalizing factors that are not relevant

- factor marginalization



In [3]:
t2=df.groupby(['bronc','smoke'],as_index=False)
t3=t2['probability'].sum()
t3

,bronc,smoke,probability
0,no,no,0.35
1,no,yes,0.20
2,yes,no,0.15
3,yes,yes,0.30


In [4]:
df.loc[(df["smoke"] == 'no') & (df["bronc"] == 'no'), :].head()

,dysp,xray,either,tub,lung,bronc,smoke,asia,probability
6,yes,yes,yes,yes,yes,no,no,yes,0.000001
7,yes,yes,yes,yes,yes,no,no,no,0.000024
14,yes,yes,yes,yes,no,no,no,yes,0.000119
15,yes,yes,yes,yes,no,no,no,no,0.002353
22,yes,yes,yes,no,yes,no,no,yes,0.000023


In [5]:
df.loc[(df["bronc"] == 'no') & (df["smoke"] == 'no'), "probability"].sum()

0.35

### Factor reduction to filter evidence

In [6]:
t4=t3.loc[ (t3["smoke"] == 'yes'), :]
t4

,bronc,smoke,probability
1,no,yes,0.2
3,yes,yes,0.3


In [7]:
psum=t4['probability'].sum()
t4['probability']=t4['probability']/psum
t4

/Users/SungwonLyu/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,bronc,smoke,probability
1,no,yes,0.4
3,yes,yes,0.6


In [8]:
t5=t3.groupby(['bronc'],as_index=False)
t5['probability'].sum()

,bronc,probability
0,no,0.55
1,yes,0.45


#### Short comings of the brute-force approach

#### Using the Variable Elimination approach

- Factors

Factor product / Factor Marginalization / Factor Reductions

Scope

- Variable Elimination

1) Multiplying factors

2) Marginalizing a variable (which is in the scope of all multiplied factors)

3) Producing a new factor


In [9]:
asia=TableCPDFactor("asia",bn)
phi_1=TableCPDFactor("tub",bn)
phi_1.multiplyfactor(asia)
printdist(phi_1,bn)

,asia,tub,probability
0,yes,yes,0.0005
1,yes,no,0.0095
2,no,yes,0.0099
3,no,no,0.9801


In [10]:
phi_1.sumout("asia")
printdist(phi_1,bn)

,tub,probability
0,yes,0.0104
1,no,0.9896


In [11]:
phi_2=TableCPDFactor("smoke",bn)
[phi_2.multiplyfactor(TableCPDFactor(i,bn)) for i in ["lung","bronc"]]
phi_2.sumout("smoke")
printdist(phi_2,bn)

,bronc,lung,probability
0,yes,yes,0.0315
1,yes,no,0.4185
2,no,yes,0.0235
3,no,no,0.5265


In [12]:
phi_3=TableCPDFactor("either",bn)
phi_3.multiplyfactor(phi_1)
phi_3.sumout("tub")
printdist(phi_3,bn)

,lung,either,probability
0,yes,yes,1.0000
1,yes,no,0.0000
2,no,yes,0.0104
3,no,no,0.9896


In [13]:
phi_4=phi_3
phi_4.multiplyfactor(phi_2)
phi_4.sumout("lung")
print ("variables in scope ",phi_4.scope)

variables in scope  ['either', 'bronc']


In [14]:
phi_5=TableCPDFactor("xray",bn)
phi_5.multiplyfactor(phi_4)
phi_5.multiplyfactor(TableCPDFactor("dysp",bn))
phi_5.sumout("either")
print ("variables in scope ",phi_5.scope)

variables in scope  ['xray', 'bronc', 'dysp']


In [15]:
phi_6=phi_5
phi_6.sumout("dysp")
printdist(phi_6,bn)

,bronc,xray,probability
0,yes,yes,0.055843
1,yes,no,0.394157
2,no,yes,0.054447
3,no,no,0.495553


In [16]:
phi_6.reducefactor("xray",'yes')
printdist(phi_6,bn)

,bronc,probability
0,yes,0.055843
1,no,0.054447


In [17]:
summ = sum(phi_6.vals)
phi_6.vals=[i/float(summ) for i in phi_6.vals]
printdist(phi_6,bn)

,bronc,probability
0,yes,0.506326
1,no,0.493674


In [18]:
bn = loadbn("asia1.txt")
evidence = {"xray":'yes'}
query = {"bronc":'yes'}
fn = TableCPDFactorization(bn)
result = fn.condprobve(query, evidence)
printdist(result,bn)

,bronc,probability
0,yes,0.506326
1,no,0.493674


### Complexity of Variable Elimination

- Why does elimination ordering matter?
---

### Graph perspective

Factor 와 Graph는 같은 내용, 다른 표현 방법

이러한 변수 제거방법이 그래프에 어떻게 영향을 미칠까

변수 제거방법은 Directed나 Undirected나 똑같이 영향을 미치지 때문에, Undirected라고 가정하고 진행한다. 

---

- moralization

![Image](figures/1.png)

두가지를 연결하고 중간을 지우는 것(새로운 Factor)은 그 둘 사이에 직접적인 링크를 만드는 것과 같다.

새로운 Factor은 새로운 종속관계를 의미하기 때문에 그래프에도 나타나야 한다. 

이러한 Moralization의 결과 나타난 네트워크가 Induced Markov network이다. 

- minimal I-map

이렇게 새로운 연결을 더하는 것을 fill edges라고 함

각 Factor에 대응하여 완전하게 연결된 그래프가 minimal I-map.

P분포에서 이렇게 모두 다 연결하면 Graph G

• G is an I-map of P
• If G ' ⊂ G and G ' is not an I-map of P

다시 말하면 I-map은 이러한 연결을 모두 하고도 독립상태인 노드들을 보여준다. 

엣지를 더하는 것은 변수 제거 순서에 따른다.

---

#### Learning the induced width from the graph structure

![Image](figures/2.png)

clique: 모두 연결된 하부그래프 ex)BCD

- Why does induced width matter?

induced width: number of nodes in the largest clique minus one ex)2

minimal induced width: VE알고리즘 후에 (다 연결하는것은 아님)최소 induced width

induced graph의 cluques는 VE 알고리즘의 런타임을 근사할 수 있게 해준다. 

최고의 VE순서를 찾는다고 해도 그 시간은 지수적으로 증가할 수 있다. 그렇기 때문에 추정하는 방법을 찾아야한다.

- Finding VE orderings

Greedy algorithms이 좋다. 

Several cost functions도 쓰일수 도 있다. 

ex) choosing the smallest factor first (a node with the least number of neighbors)

## 3) The tree algorithm

개념

- A cluster graph: 군집 그래프 - factor과 비슷하다

- message passing: 군집사이의 메시지를 전하는 것. 

ex) 파티에서 셸리와 클레어가 만나는데 셸리는 BCD를 알고있고 클레어는 DEF를 알고있다. 그들은 공통된 친구인 D에 대한 이야기를 나눌 수 있다. 

- Separation Set (sepset) : 두 클러스터 사이의 공통된 변수

ex) {Shelly, B, C, D} 와 {Clair, D, E, f}의 sepset은 {D}

- message passing 알고리즘은 여러종류가 있지만 clique tree- message passing algorithm (junction tree algorithm)를 알아보기로 한다.

### The four stages of the juction tree algorithm

### Using the junction tree algorithm for inference

1) 베이즈 네트워크가 Join tree라는 구조로 변환된다. (Join tree = Junction tree = Cluster tree = Clique tree) 

![Image](figures/3.png)

#### Stage 1.1 – moralization

![Image](figures/4.png)

- 우선 모든 방향성있는 엣지를 없는 것으로 바꾼다. 

- V구조를 가지고 있는 노드의 부모들을 모두 연결한다.(moralization)

#### Stage 1.2 – triangulation

![Image](figures/5.png)

- 이렇게 이어서 triangulated graph 구조를 만든다.

- 이 triangulated graph의 subset을 밝힌다 (Clique)

#### Stage 1.3 – building the join tree

![Image](figures/6.png)

- clique를 클러스터로 시작해서 Join tree를 만든다. 

- JOin tree: 한 노드가 두 클리크를 연결하면 클리크의 경로의 모든 노드에 모두 나타나야 한다. 

- the variables in the cliques {Bronc, Either, Smoke} and {Bronc, Dysp, Either} are the variables in the {Bronc, Either} sepset.

#### Stage 2 – initializing potentials

각 클러스터의 가능성(potential)이 초기화된다. 

확률(probability)와 비교하여 Undirected Marcov Chain이기 때문에 모두 더하여 1이 아니고 가능성이라고 표현단다.

가능성이란 CPD나 표와 유사하게 각 할당마다 값이 있다. 

클러스터와 sepset는 각각 가능성이 있다. 

#### Stage 3 – message passing

![Image](figures/7.png)

이웃한 클러스터 사이에 메시지를 전달한다.

각 메시지는 클러스터가 특정한 변수에 대하여 가지고 있는 정보를 포함한다. 

각 메시지는 동시에 전달되지만 다른 클러스터에 전달하기 전에 전달 받아야 한다.

나무 구조로 생각하면 위 방향으로, 아래 방향으로 두 단계로 일어난다.

the source cluster

intervene sepset

destination cluster

각 클러스터의 sepset이 일관된 값을 가지면 메시지전달이 완료된다. 

예시)

sepset potential: φr

destination cluster potential: φy

sepset할당

![Image](figures/8.png)

destination 할당

![Image](figures/9.png)

=> Consistent belief

#### Stage 4 - Marginalization

다른 변수들에 대하여 marginalization한 후 

어떤 변수에 대해서도 query할 수 있다. 

![Image](figures/10.png)

나머지 변수들에 대해 Marginal out 하면

![Image](figures/11.png)

다른 클러스터에서 marginal out 해도 같은 값을 가질까?

![Image](figures/12.png)

그렇다고 한다. 

---

- 장점: marginal queries에 대해 한번의 계산으로 빠르게 나타낼 수 있따.

- 단점: 공간을 많이 차지한다. VE는 중간과정을 지우지만, junction tree는 모두 저장한다. 